In [ ]:
import numpy as np
import sys
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
import math
import ROOT
from scipy.integrate import dblquad

sys.path.append('/Users/alexanderantonakis/Desktop/Software/CEvNS_RATES/Classes')
from snsgen import SNSGen

print("Finished importing")

# Initialize the SNS Generator for Ar40: 1 Ton, 20 m Baseline

In [ ]:
# initialize the SNS generator with the material of choice

mdet = 1000 #kg
L = 20. # meters
pot = 2.1*(10**23)

material = "Ar40"
#material = "He"

gen = SNSGen(mdet, L, pot, material)

print("Set up the SNS generator for our config")

# Spectral Functions for the Neutrino Flux

In [ ]:
Es = np.arange(0, 53, 0.001)
s_nue = gen.spectral_nue(Es)
s_numubar = gen.spectral_numubar(Es)
s_numu = gen.spectral_numu(Es)
plt.plot(Es, s_nue, label=r"$\nu_{e}$")
plt.plot(Es, s_numubar, label=r"$\bar{\nu}_{\mu}$")
plt.plot(Es, s_numu, label=r"$\nu_{\mu}$")
plt.legend(fontsize=14)
plt.xlabel("Neutrino Energy [MeV]", fontsize=14)
plt.ylabel("Spectral Shape [Arb.]", fontsize=14)
plt.show()

# Calculate the normalized flux / POT

In [ ]:
f_nue = gen.flux_nue(Es)
f_numubar = gen.flux_numubar(Es)
f_numu = gen.flux_numu(Es)
plt.scatter([], [], label="Detector Baseline: " + str(gen.L)+" m")
plt.plot(Es, f_nue, label=r"$\nu_{e}$")
plt.plot(Es, f_numubar, label=r"$\bar{\nu}_{\mu}$")
plt.plot(Es, f_numu, label=r"$\nu_{\mu}$")
plt.legend(fontsize=14)
plt.xlabel("Neutrino Energy [MeV]", fontsize=14)
plt.ylabel("Neutrino Flux [" + r"$\nu$"+"/"+"$m^2$"+"/POT]", fontsize=14)
plt.yscale("log")
plt.show()

# Plot the Helm Form Factor used in the Cross-section

In [ ]:
Es = np.arange(0.001, 0.14+0.001, 0.001)
qs = np.sqrt(2*gen.M*Es)
fs = gen.FH(Es)**2
plt.plot(qs, fs)
plt.xlabel("Momentum Transfer [MeV]", fontsize=14)
plt.ylabel(r"Helm Form Factor Squared", fontsize=14)
plt.show()

# Calculate the Cross-section using the Differential Cross-section

In [ ]:
Enus = np.linspace(0.00001, 53, 1000)
sigmas = gen.Sigma(Enus)


plt.plot(Enus, sigmas*(100**2)*(10**38))
plt.xlim([5, 53])
plt.ylim([10**(-5), 10**2])
plt.yscale("log")
plt.ylabel("Cross-section ("+r"$10^{-38} cm^2$"+")", fontsize=14)
plt.xlabel("Neutrino Energy [MeV]", fontsize=14)
plt.show()

# Event Rate as a function of Recoil Energy

In [ ]:
Ers = np.arange(0.001, 0.14, 0.001)
plt.plot(Ers*1000, gen.N_Er_numubar(Ers), label=r"$\bar{\nu}_{\mu}$")
plt.plot(Ers*1000, gen.N_Er_nue(Ers), label=r"$\nu_{e}$")
plt.plot(Ers*1000, gen.N_Er_numu(Ers), label=r"$\nu_{\mu}$")
plt.ylabel("Events/POT/keV", fontsize=14)
plt.xlabel("Nuclear Recoil Energy [keV]", fontsize=14)
plt.legend(fontsize=14)
plt.xlim([1, 140])
#plt.ylim([0, 350])
plt.show()

In [ ]:
Ers = np.arange(0.001, 0.14, 0.001)
plt.plot(Ers*1000, gen.pot*gen.N_Er_numubar(Ers), c="r", label=r"$\bar{\nu}_{\mu}$")
plt.plot(Ers*1000, gen.pot*gen.N_Er_nue(Ers), c="g", label=r"$\nu_{e}$")
plt.plot(Ers*1000, gen.pot*gen.N_Er_numu(Ers), c="b", label=r"$\nu_{\mu}$")
plt.ylabel("Events/year/keV", fontsize=14)
plt.xlabel("Nuclear Recoil Energy [keV]", fontsize=14)
plt.legend(fontsize=14)
plt.xlim([1, 150])
plt.ylim([0, 315])
plt.show()

# Plot the Differential Recoil Spectrum / POT

In [ ]:
Ers = np.linspace(0.001, 0.16, 1000)
costhetas = np.linspace(0.01, 1, 1000)


X, Y = np.meshgrid(Ers, costhetas)  
Z = gen.dN_dEr_dcos_muon(X, Y)

# Create a contour plot
plt.figure(figsize=(8, 6))

# Contour lines
#contour = plt.contour(X*1000, Y, Z, levels=100, cmap="viridis", norm=LogNorm())  # 20 levels
#plt.clabel(contour, inline=True, fontsize=8)               # Label the contours

# Filled contour
contour = plt.contourf(X*1000, Y, Z, levels=100, 
                       cmap="viridis", alpha=0.7, norm=LogNorm())  # Filled contours

colorbar = plt.colorbar(contour)
colorbar.set_label(r"$\frac{d^{2}N}{dE_{r}d\Omega_{r}}$" + "/POT", fontsize=14)

plt.scatter([], [], c="black", label="Detector Mass: "+str(gen.mdet)+ " kg")
plt.scatter([], [], c="black", label="Baseline: "+str(gen.L)+ " m")

#plt.xlim([0, 150])
plt.ylim([0, 1.1])
# Add labels
#plt.title("Contour Plot Example")
plt.xlabel("Nuclear Recoil Energy [keV]", fontsize=14)
plt.ylabel("cos"+r"$\theta_{r}$", fontsize=14)
plt.xscale("log")
# Show the plot
plt.xlim([0, 200])
plt.legend(loc="lower right")
plt.title(r"$\nu_{e} + \bar{\nu}_{\mu}$"+" Flux Contribution", fontsize=20)
plt.show()

In [ ]:
Ers = np.linspace(0.001, 0.16, 1000)
costhetas = np.linspace(0.01, 1, 1000)


X, Y = np.meshgrid(Ers, costhetas)  
Z = gen.dN_dEr_dcos_muon(X, Y)

# Create a contour plot
plt.figure(figsize=(8, 6))



# Filled contour
contour = plt.contourf(X*1000, Y, Z*gen.pot, levels=100, 
                       cmap="viridis", alpha=0.7, norm=LogNorm())  # Filled contours

colorbar = plt.colorbar(contour)
colorbar.set_label(r"$\frac{d^{2}N}{dE_{r}d\Omega_{r}}$", fontsize=14)

plt.scatter([], [], c="black", label="Detector Mass: "+str(gen.mdet)+ " kg")
plt.scatter([], [], c="black", label="Baseline: "+str(gen.L)+ " m")

#plt.xlim([0, 150])
plt.ylim([0, 1.1])
# Add labels
#plt.title("Contour Plot Example")
plt.xlabel("Nuclear Recoil Energy [keV]", fontsize=14)
plt.ylabel("cos"+r"$\theta_{r}$", fontsize=14)
plt.xscale("log")
# Show the plot

min_costheta = gen.get_min_cos(Ers)

plt.plot(Ers*1000, min_costheta, c="r")

plt.xlim([1, 200])
plt.legend(loc="lower right")
plt.title(r"$\nu_{e} + \bar{\nu}_{\mu}$"+" Flux Contribution", fontsize=20)
plt.show()

In [ ]:
Ers = np.linspace(0.001, 1.5, 1000)
costhetas = np.linspace(0.01, 1, 1000)


X, Y = np.meshgrid(Ers, costhetas)  
Z = gen.dN_dEr_dcos_muon(X, Y)

# Create a contour plot
plt.figure(figsize=(8, 6))



# Filled contour
contour = plt.contourf(X*1000, Y, Z*gen.pot, levels=100, 
                       cmap="viridis", alpha=0.7, norm=LogNorm())  # Filled contours

colorbar = plt.colorbar(contour)
colorbar.set_label(r"$\frac{d^{2}N}{dE_{r}d\Omega_{r}}$", fontsize=14)

plt.scatter([], [], c="black", label="Detector Mass: "+str(gen.mdet)+ " kg")
plt.scatter([], [], c="black", label="Baseline: "+str(gen.L)+ " m")

#plt.xlim([0, 150])
plt.ylim([0, 1.1])
# Add labels
#plt.title("Contour Plot Example")
plt.xlabel("Nuclear Recoil Energy [keV]", fontsize=14)
plt.ylabel("cos"+r"$\theta_{r}$", fontsize=14)
plt.xscale("log")
# Show the plot

min_costheta = gen.get_min_cos(Ers)

plt.plot(Ers*1000, min_costheta, c="r")

plt.xlim([1, 1500])
plt.legend(loc="lower right")
plt.title(r"$\nu_{e} + \bar{\nu}_{\mu}$"+" Flux Contribution", fontsize=20)
plt.show()

In [ ]:
# Crude Approximation

Ers = np.arange(0.001, 1.5, 1./1000)
costhetas = np.arange(0.01, 1.01, 0.01)


X, Y = np.meshgrid(Ers, costhetas)  
Z = gen.dN_dEr_dcos_muon(X, Y)


Scale = 2*math.pi*(Ers[1] - Ers[0])*(costhetas[1] - costhetas[0])
#Scale = (Ers[1] - Ers[0])*(costhetas[1] - costhetas[0])
N = Z*gen.pot*Scale

mask = (N < 10**-2)

N[mask] = 0

print(np.max(N))

plt.figure(figsize=(8, 6))

# Filled contour
#contour = plt.contourf(X*1000, Y, N, levels=[10**-2, 10**-1, 1], 
#                       cmap="viridis", alpha=0.7, norm=LogNorm())  # Filled contours

contour = plt.contour(X*1000, Y, N, levels=[10**-2, 10**-1, 1, 2], cmap="viridis")  # contours

colorbar = plt.colorbar(contour)
colorbar.set_label(r"Events/year", fontsize=14)

plt.scatter([], [], c="black", label="Detector Mass: "+str(gen.mdet)+ " kg")
plt.scatter([], [], c="black", label="Baseline: "+str(gen.L)+ " m")

#plt.xlim([0, 150])
plt.ylim([0, 1])
# Add labels
#plt.title("Contour Plot Example")
plt.xlabel("Nuclear Recoil Energy [keV]", fontsize=14)
plt.ylabel("cos"+r"$\theta_{r}$", fontsize=14)
plt.xscale("log")
# Show the plot

min_costheta = gen.get_min_cos(Ers)

#plt.plot(Ers*1000, min_costheta, c="r")

plt.xlim([1, 1500])
plt.legend(loc="lower right")
plt.title(r"$\nu_{e} + \bar{\nu}_{\mu}$"+" Flux Contribution", fontsize=20)
plt.show()

# Crude Approximation

In [ ]:
# Crude Approximation

Ers = np.arange(0.0001, 1.5, 1./10000)
costhetas = np.arange(0.001, 1.001, 0.001)


X, Y = np.meshgrid(Ers, costhetas)  
Z = gen.dN_dEr_dcos_muon(X, Y)


Scale = 2*math.pi*(Ers[1] - Ers[0])*(costhetas[1] - costhetas[0])
Scale = (Ers[1] - Ers[0])*(costhetas[1] - costhetas[0])
N = Z*gen.pot*Scale

mask = (N < 10**-3)

#N[mask] = 0

print(np.max(N))

plt.figure(figsize=(8, 6))

# Filled contour
contour = plt.contourf(X*1000, Y, N, levels=1000, 
                      cmap="viridis", alpha=0.7, norm=LogNorm())  # Filled contours

#contour = plt.contour(X*1000, Y, N, levels=[10**-4, 10**-3, 10**-2], cmap="viridis")  # contours

colorbar = plt.colorbar(contour)
colorbar.set_label(r"Events/year", fontsize=14)

plt.scatter([], [], c="black", label="Detector Mass: "+str(gen.mdet)+ " kg")
plt.scatter([], [], c="black", label="Baseline: "+str(gen.L)+ " m")

#plt.xlim([0, 150])
plt.ylim([0, 1])
# Add labels
#plt.title("Contour Plot Example")
plt.xlabel("Nuclear Recoil Energy [keV]", fontsize=14)
plt.ylabel("cos"+r"$\theta_{r}$", fontsize=14)
plt.xscale("log")
# Show the plot

min_costheta = gen.get_min_cos(Ers)
print(min_costheta)

#plt.plot(Ers*1000, min_costheta, c="r")

plt.xlim([1, 1500])
plt.legend(loc="lower right")
plt.title(r"$\nu_{e} + \bar{\nu}_{\mu}$"+" Flux Contribution", fontsize=20)
plt.show()

# PLot Recoil Spectrum from GenerateRecoilSpectrum.py in Main

## Need to Check if off by a factor of 2pi or 4pi from the integration measure

In [ ]:
rfile = ROOT.TFile.Open("Main/Ar_spectrum.root", "READ")
h_multi_Ar = rfile.Get("N_2D")
h_multi_Ar.SetDirectory(0)
rfile.Close()

In [ ]:
c = ROOT.TCanvas("c", "c", 700, 500)
c.SetLogz()
c.SetLogx()
h_multi_Ar.SetStats(0)
h_multi_Ar.SetTitle("Argon Recoil Spectrum")
h_multi_Ar.GetXaxis().SetTitle("Nuclear Recoil Energy [keV]")
h_multi_Ar.GetXaxis().SetRangeUser(1, 150)
h_multi_Ar.GetYaxis().SetTitle("cos#theta_{r}")
h_multi_Ar.GetZaxis().SetTitle("Interactions/Year")
h_multi_Ar.GetXaxis().SetTitleOffset(1.5)
h_multi_Ar.GetZaxis().SetTitleOffset(0.7)
h_multi_Ar.Draw("Colz")
c.Draw()

# Slow Implementation of the Integration Beyond this Point

In [ ]:
Stop

In [ ]:

def N(Er1, Er2, cos1, cos2):
    def f(y, x):
        return 4*math.pi*gen.dN_dEr_dcos_muon(y, x)
        
    v = dblquad(f, cos1, cos2, Er1, Er2)
    #print(v)
    return v[0]

print(N(1/1000, 10/1000, 0.2, 0.6)*gen.pot)


# Method with Double Integration --> Slow

In [ ]:
# Crude Approximation

Ers = np.arange(0.001, 1.5, 1./1000)
costhetas = np.arange(0.01, 1.01, 0.01)

Ermin = 1
Ermax = 1500

nbinsx = Ermax-Ermin

cosmin = 0.01
cosmax = 1

coswidth = 0.01
nbinsy = int((cosmax - cosmin)/coswidth)


h = ROOT.TH2D("N_2D", "", nbinsx, Ermin, Ermax, nbinsy, cosmin, cosmax)

print("Start Loop Over", nbinsx*nbinsy, " Cells")
count = 0
for i in range(1, h.GetNbinsX()+1):
    for j in range(h.GetNbinsY()+1):
        Er1 = h.GetXaxis().GetBinLowEdge(i)
        Er2 = Er1 + h.GetXaxis().GetBinWidth(i)
        cos1 = h.GetYaxis().GetBinLowEdge(j)
        cos2 = cos1 + h.GetYaxis().GetBinWidth(j)
        v = N(Er1/1000, Er2/1000, cos1, cos2)*gen.pot
        h.SetBinContent(i, j, v)
        count += 1
        if count % 10000 == 0:
            print("Cell", count)

c = ROOT.TCanvas("c", "c", 700, 500)
c.SetLogz()
h.Draw("Colz")
c.Draw()

In [ ]:
c = ROOT.TCanvas("c", "c", 700, 500)
c.SetLogz()
c.SetLogx()
h.SetStats(0)
h.GetXaxis().SetTitle("Nuclear Recoil Energy [keV]")
h.GetYaxis().SetTitle("cos#theta_{r}")
h.GetZaxis().SetTitle("Interactions/Year")
h.GetXaxis().SetTitleOffset(1.5)
h.GetZaxis().SetTitleOffset(0.7)
h.Draw("Colz")
c.Draw()

In [ ]:
contours = [0.01, 0.1, 1]  # set your desired contour levels
h.SetContour(len(contours))
for i, level in enumerate(contours):
    h.SetContourLevel(i, level)

# Draw the histogram as a contour plot
c = ROOT.TCanvas("c", "Contour Plot", 700, 500)
c.SetLogz()
c.SetLogx()
h.Draw("CONT1Z")  # "CONT1Z" adds colored contours + axis

c.Update()
c.Draw()

In [ ]:
contours = [0.01, 0.1, 1]  # set your desired contour levels
h.Scale(1/(2*math.pi))
h.SetContour(len(contours))
for i, level in enumerate(contours):
    h.SetContourLevel(i, level)

# Draw the histogram as a contour plot
c = ROOT.TCanvas("c", "Contour Plot", 700, 500)
c.SetLogz()
c.SetLogx()
h.Draw("CONT1Z")  # "CONT1Z" adds colored contours + axis

c.Update()
c.Draw()

In [ ]:
print(h.GetMaximum())

In [ ]:
contours = [0.01, 0.1, 1]  # set your desired contour levels
h.SetContour(len(contours))
for i, level in enumerate(contours):
    h.SetContourLevel(i, level)

# Draw the histogram as a contour plot
c = ROOT.TCanvas("c", "Contour Plot", 700, 500)
c.SetLogz()
c.SetLogx()
h.Draw("CONT1Z")  # "CONT1Z" adds colored contours + axis

c.Update()
c.Draw()